In [1]:

from PollenDataAnalyzer import PollenDataAnalyzer
# %pip install tqdm
from collections import defaultdict

In [2]:

# %pip install matplotlib
# from PollenDataAnalyzer import PollenDataAnalyzer

# TODOS !!!!
# should we change the itensity_mapping, it seem kind of werid that it's all low and high
# 


In [3]:
LOCAL_PATH = "../Project data/"

In [4]:
#!!!download and Change the data_path for your own "dataset-for-roi/cleaned_status_intensity_observation_data.csv" path on your local computer
#also download and change the land_cover_path

analyzer = PollenDataAnalyzer(
    data_path= LOCAL_PATH + "status_intensity_observation_data.csv",  # or the raw CSV
    mapping_path="../intensity_mapping.json",
    Phenophase_path="../Phenophase.json", 
    land_cover_path = LOCAL_PATH + "table_2021_counties_max.xlsx",
    project_data_path = LOCAL_PATH
)

In [5]:
analyzer.map_phenophase_categories()

#first 2 rows
analyzer.df["Phenophase_Category"].head(5)

0    Reproductive
1    Reproductive
2    Reproductive
3    Reproductive
4    Reproductive
Name: Phenophase_Category, dtype: object

In [8]:
#using the pollen_df to convert only the correct phenophase data with corresponding lag_long to county ideas
#https://geo.fcc.gov/api/census/#!/area/get_area
#this will take 1-3 minutues max
# and also store a copy to Kathy's personal path (LINE 169 to delete it or change it to your own path)
analyzer.lat_long_to_county()

Fetching county FIPS codes...


Fetching FIPS: 100%|███████████████████████████████████| 1671618/1671618 [00:24<00:00, 67172.94it/s]


Finished fetching county FIPS codes.
         Observation_ID   Latitude  Longitude  Species_ID  Individual_ID  \
0                 42749  37.545090 -75.927589          36            907   
1                 42749  37.545090 -75.927589          36            907   
2                 42744  37.545090 -75.927589          36            907   
3                 42744  37.545090 -75.927589          36            907   
4                 42739  37.545090 -75.927589          36            907   
...                 ...        ...        ...         ...            ...   
1671613        52137295  38.883358 -77.106964        1653         364153   
1671614        52137296  38.883358 -77.106964        1653         364153   
1671615        52137297  38.883358 -77.106964        1653         364153   
1671616        52137300  38.883358 -77.106964        1653         364153   
1671617        52137298  38.883358 -77.106964        1653         364153   

         Phenophase_ID Phenophase_Category        

In [9]:
analyzer.add_land_cover_info()

Land cover types added using hash table.
  county_fips   land_cover_type
0       51131  Cultivated crops
1       51131  Cultivated crops
2       51131  Cultivated crops
3       51131  Cultivated crops
4       51131  Cultivated crops


In [10]:
# finding the number of data points for each year

year_counts = defaultdict(int)

# Loop through each date and extract the year
for date in analyzer.df["Observation_Date"]:
    year = str(date)[:4]  # Get the year part from YYYY-MM-DD
    year_counts[year] += 1

# Sort by year
sorted_years = sorted(year_counts.items())

# Print as a clean table
print(f"{'Year':<10} {'Count':<10}")
print("-" * 20)
for year, count in sorted_years:
    print(f"{year:<10} {count:<10}")

#check if it match with Pollen-only dataset created with 31935 rows. it does!
print("final data sets", len(analyzer.df), "rows.") 


Year       Count     
--------------------
2000       19        
2001       749       
2006       20        
2009       253       
2010       255       
2011       3302      
2012       12966     
2013       39927     
2014       48343     
2015       49666     
2016       111549    
2017       105471    
2018       124559    
2019       161209    
2020       99650     
2021       249501    
2022       238439    
2023       219212    
2024       146395    
2025       60133     
final data sets 1671618 rows.


In [11]:
print(analyzer.df.columns)


# Make a copy of the column names
columns_to_getrid = analyzer.df.columns.difference(
    ['ObservedBy_Person_ID', 'Latitude', 'Longitude', 'Individual_ID', 
     'Phenophase_Description', 'Phenophase_Name', 'Site_Visit_ID', 'Plant_Nickname']
)

# Reorder columns to make 'Observation_ID' the first column
columns_reordered = ['Observation_ID'] + [col for col in columns_to_getrid if col != 'Observation_ID']

# Create a new DataFrame with the reordered columns
cleaned_df = analyzer.df[columns_reordered]

# Save the new DataFrame to the specified path
cleaned_df.to_csv(
    LOCAL_PATH + "cleaned_V2.0_status_intensity_observation_data.csv",
    index=False
)


Index(['Observation_ID', 'Latitude', 'Longitude', 'Species_ID',
       'Individual_ID', 'Phenophase_ID', 'Phenophase_Category',
       'Phenophase_Name', 'Phenophase_Definition_ID', 'Observation_Date',
       'Day_of_Year', 'Intensity_Category_ID', 'Site_Visit_ID', 'AGDD', 'Tmax',
       'Tmin', 'Prcp', 'Accum_Prcp', 'Daylength', 'county_fips',
       'land_cover_type'],
      dtype='object')
